<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/git_teste_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa pacotes que serão usados

In [ ]:
#!rm -rf cassandra*

In [ ]:
import pandas as pd
import os

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try: 
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')
  
def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit 
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try: 
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta cassandra
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio cassandra
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta cassandra
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto cassandra
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto cassandra
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do cassandra
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1]) 
      lista_comment.append(each[2]) 
      lista_code.append(each[3])  
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))	
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))	
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [ ]:
instala_cloc()

['', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 0%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 4%', '', 'Reading package lists... 45%', '', 'Reading package lists... 45%', '', 'Reading package lists... 46%', '', 'Reading package lists... 46%', '', 'Reading package lists... 56%', '', 'Reading package lists... 56%', '', 'Reading package lists... 64%', '', 'Reading package lists... 64%', '', 'Reading package lists... 67%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 69%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 70%', '', 'Reading package lists... 79%', '', 'Reading package lists... 79%', '', 'Reading package lists... 86%', '', 'Reading package lists... 86%'

# Clona repositorio principal

In [ ]:
clona_repositorio(url='https://github.com/apache/cassandra.git')
print(f"Primeiro commit do cassandra: {get_primeiro_commit('cassandra')}")
print(f"Ultimo commit do cassandra: {get_ultimo_commit('cassandra')}")

Clonando o repositório https://github.com/apache/cassandra.git
git clone https://github.com/apache/cassandra.git
concluido com sucesso!


['4e3447ae1727a0d01c9d81792acbdeeef94d86d2']

# Cria um repositorio apontando para o 1o. commit

In [ ]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_c1')
# git checkout $( git rev-list --all | tail -n 1 )
#!cd $REPOSITORIO_INICIAL && git checkout 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85
primeiro_commit = get_primeiro_commit(repositorio='cassandra')
checkout_n_commit('cassandra_c1', primeiro_commit[0])
df_info_repo = gera_df_info_repositorio(pasta_n_commit='cassandra_c1')
df_dados_linguagens = gera_df_dados(pasta_n_commit='cassandra_c1')

cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT
Checkout do repositorio no commit 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85 realizado com sucesso!


# Cria um repositório apontando para o último commit

In [ ]:
# cria pasta o primeiro/ultimo commit
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_cf')
# pega o primero/ultimo commit
ultimo_commit = get_ultimo_commit(repositorio='cassandra')
# faz o checkout para primeiro/ultimo commit
checkout_n_commit('cassandra_cf', ultimo_commit[0])
# gera dataframe de info do repositorio
df_info_repo_cf = gera_df_info_repositorio(pasta_n_commit='cassandra_cf')
# gera dataframe de dados linguagem do repositorio
df_dados_linguagens_cf = gera_df_dados(pasta_n_commit='cassandra_cf')

cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT
Checkout do repositorio no commit 4e3447ae1727a0d01c9d81792acbdeeef94d86d2 realizado com sucesso!


# Gera dataframe com os dados dos repositorios

In [ ]:
df_repo = gera_df_dados_repositorios(df_info_repo, df_info_repo_cf)
df_repo

,descricao,dados_c1,dados_cf
0,nome,cassandra_c1,cassandra_cf
1,qtd_arquivos,369,5186
2,qtd_commits,1,28432
3,tamanho_pasta_mb,407,486
4,qtd_colaboradores_commits,1,578
5,data_primeiro_commit,2009-03-02,2009-03-02
6,data_ultimo_commit,2009-03-02,2023-03-10
